In [179]:
import os
from PIL import Image
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
from glob import glob
import torchvision
import tqdm

In [180]:
class customdataset(Dataset): 
    def __init__(self, data_dir, transform= None): 
        #이닛 해주고
        self.data_dir= data_dir 
        self.transform= transform
        #이미지들 경로명 담을 리스트
        self.image_paths= []
        self.labels= []

        #이제 여기서 데이터를 결정해줘야해. 
        #경로에서 'NORMAL' 폴더에 있는 하위 파일들을 모두 리스트로 만듦
        # normal_dir = os.path.join(data_dir, 'NORMAL')
        # for img in glob(f"{normal_dir}/*"): 
        #     self.image_paths.append(os.path.join(normal_dir, img))
        #     self.labels.append(0)

        # pneumonia_dir = os.path.join(data_dir, 'PNEUMONIA')
        # for img in glob(f"{pneumonia_dir}/*"): 
        #     self.image_paths.append(os.path.join(pneumonia_dir, img))
        #     self.labels.append(1)

        # NORMAL 폴더의 이미지 경로와 라벨 (0)
        normal_dir = os.path.join(data_dir, 'NORMAL')
        for img_name in os.listdir(normal_dir):
            self.image_paths.append(os.path.join(normal_dir, img_name))
            self.labels.append(0)

        # PNEUMONIA 폴더의 이미지 경로와 라벨 (1)
        pneumonia_dir = os.path.join(data_dir, 'PNEUMONIA')
        for img_name in os.listdir(pneumonia_dir):
            self.image_paths.append(os.path.join(pneumonia_dir, img_name))
            self.labels.append(1)

            
    def __len__(self): 
        #len이건 필수래 나중에 DataLoader 하려면 
        #몇번 반복해서 받을건지를 결정하는 요소? 
        #이미지파일 갯수만큼 받아야하니까
        return len(self.image_paths)

    def __getitem__(self, idx): 
        #이것도 필수래 DataLoader 에서 이터레이터한 객체를 받아올 수 있다는데? 
        #그래서 for문같은 반복문 추가 안해줘도 DataLoader에서 알아서 반복 해준대 ㅇㅇ
        #이미지 경로를 받아서 실제 이미지를 열어주는 구간
        
        image = Image.open(self.image_paths[idx]).convert('L')
        label= self.labels[idx]

        if self.transform: 
            image= self.transform(image)

        return image, label


In [181]:
train_transform= transforms.Compose([ 
    #데이터 전처리 하기위해서 필수인 과정 
    #데이터 증강도 할 수 있음 +a인 느낌 
    transforms.Resize((224, 224)),
    # transforms.Grayscale(num_output_channels=1), 
    transforms.RandomHorizontalFlip(),      # 수평 뒤집기
    transforms.RandomRotation(10),          # 10도 이내로 회전
    transforms.ColorJitter(brightness=0.2, contrast=0.2),  # 밝기 및 대비 조절
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),  # 회전 없이 이동 변환만 적용
    transforms.ToTensor(), 
    transforms.Normalize(mean=[0.5], std=[0.5]) 
])

val_transform= transforms.Compose([
    #검증 데이터셋을 만들 땐 전처리만
    #데이터 증강은 필요없음. 
    transforms.Resize((224, 224)),
    transforms.Grayscale(num_output_channels=1), 
    transforms.ToTensor(), 
    transforms.Normalize(mean=[0.5], std=[0.5]) 
])


In [182]:
full_dataset= customdataset(data_dir= 'chest_xray/train', transform= train_transform) 
train_size= int(0.8*len(full_dataset)) 
val_size= len(full_dataset) - train_size
train_dataset, val_dataset= random_split(full_dataset, [train_size, val_size]) 


In [183]:
len(full_dataset)

5216

In [184]:
print(full_dataset.transform, train_dataset.dataset.transform)

Compose(
    Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=True)
    RandomHorizontalFlip(p=0.5)
    RandomRotation(degrees=[-10.0, 10.0], interpolation=nearest, expand=False, fill=0)
    ColorJitter(brightness=(0.8, 1.2), contrast=(0.8, 1.2), saturation=None, hue=None)
    RandomAffine(degrees=[0.0, 0.0], translate=(0.1, 0.1))
    ToTensor()
    Normalize(mean=[0.5], std=[0.5])
) Compose(
    Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=True)
    RandomHorizontalFlip(p=0.5)
    RandomRotation(degrees=[-10.0, 10.0], interpolation=nearest, expand=False, fill=0)
    ColorJitter(brightness=(0.8, 1.2), contrast=(0.8, 1.2), saturation=None, hue=None)
    RandomAffine(degrees=[0.0, 0.0], translate=(0.1, 0.1))
    ToTensor()
    Normalize(mean=[0.5], std=[0.5])
)


In [185]:

# 검증 데이터는 데이터 증강을 적용하지 않기 때문에 변환을 따로 지정
val_dataset.dataset = customdataset(data_dir='chest_xray/train', transform=val_transform)


In [186]:
print(full_dataset.transform, train_dataset.dataset.transform, val_dataset.dataset.transform)

Compose(
    Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=True)
    RandomHorizontalFlip(p=0.5)
    RandomRotation(degrees=[-10.0, 10.0], interpolation=nearest, expand=False, fill=0)
    ColorJitter(brightness=(0.8, 1.2), contrast=(0.8, 1.2), saturation=None, hue=None)
    RandomAffine(degrees=[0.0, 0.0], translate=(0.1, 0.1))
    ToTensor()
    Normalize(mean=[0.5], std=[0.5])
) Compose(
    Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=True)
    RandomHorizontalFlip(p=0.5)
    RandomRotation(degrees=[-10.0, 10.0], interpolation=nearest, expand=False, fill=0)
    ColorJitter(brightness=(0.8, 1.2), contrast=(0.8, 1.2), saturation=None, hue=None)
    RandomAffine(degrees=[0.0, 0.0], translate=(0.1, 0.1))
    ToTensor()
    Normalize(mean=[0.5], std=[0.5])
) Compose(
    Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=True)
    Grayscale(num_output_channels=1)
    ToTensor()
    Normalize(mean=[0.5], std=[0.5

In [187]:
# DataLoader 설정
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=4)


In [188]:
class EnhancedCNN(nn.Module): 
    def __init__(self): 
        #초기화, 변수 설정해주는 구간 
        super(EnhancedCNN, self).__init__()
        self.conv1= nn.Conv2d(1, 32, kernel_size= 3, stride= 1, padding= 1) 
        self.bn1= nn.BatchNorm2d(32) 
        self.conv2= nn.Conv2d(32, 64, kernel_size= 3, stride= 1, padding= 1) 
        self.bn2= nn.BatchNorm2d(64) 
        self.conv3= nn.Conv2d(64, 128, kernel_size= 3, stride= 1, padding= 1) 
        self.bn3= nn.BatchNorm2d(128)
        self.pool= nn.MaxPool2d(kernel_size= 2, stride= 2, padding= 0) 
        self.dropout= nn.Dropout(0.5) 
        self.fc1= nn.Linear(128*28*28, 256)
        self.fc2= nn.Linear(256, 2) 

    def forward(self, x): 
        #여기가 실제로 층 쌓는 구간
        x= self.pool(F.relu(self.bn1(self.conv1(x))))
        x= self.pool(F.relu(self.bn2(self.conv2(x)))) 
        x= self.pool(F.relu(self.bn3(self.conv3(x)))) 
        x= x.view(-1, 128*28*28)
        x= F.relu(self.fc1(x)) 
        x= self.dropout(x) 
        x= self.fc2(x) 
        return x

In [190]:
device= torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model= EnhancedCNN().to(device) 
criterion= nn.CrossEntropyLoss() 
optimizer= optim.SGD(model.parameters(), lr= 0.001, momentum= 0.9)
num_epochs= 10

In [191]:
for epoch in range(num_epochs): 
    #초기화 해주는거 
    #학습모드로 변경
    model.train() 
    running_loss= 0.0

    #얘네는 recall 구하기 위해서 추가한거
    all_train_labels= []
    all_train_predictions= [] 

    for images, labels in train_loader: 
        #gpu 디바이스로 적용해서 진행하려고 to(device) 
        images, labels= images.to(device), labels.to(device) 

        optimizer.zero_grad() 

        outputs= model(images) 
        loss= criterion(outputs, labels) 

        #recall 구하려고 하는거 예측값 저장
        _, predicted= torch.max(outputs.data, 1)
        all_train_labels.extend(labels.cpu().numpy())
        all_train_predictions.extend(predicted.cpu().numpy()) 

        loss.backward() 
        optimizer.step() 

        running_loss+= loss.item()
    #손실 출력 
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss / len(train_loader):.4f}')

    #recall 값 출력 
    train_report= classification_report(all_train_labels, all_train_predictions, target_names= ['NORMAL', 'PNEUMONIA'], output_dict= True) 
    train_recall= train_report['weighted avg']['recall']
    print(f'train_recall: {train_recall:.4f}')



    #val 상단에 해놓은거랑 똑같이 하면 댐
    model.eval() 
    all_val_labels= []
    all_val_predictions= [] 

    #정확도 구할 때 씀 
    correct= 0 
    total= 0 

    #
    with torch.no_grad(): 



Epoch [1/10], Loss: 0.4242


KeyboardInterrupt: 